# 如何确定输入线性层前的特征数据

一.采用nn.Sequential方法

import torch

import torch.nn as nn

model = nn.Sequential(
    
    nn.Linear(10, 20),  # 全连接层，输入维度10，输出维度20
    
    nn.ReLU()          # ReLU激活函数
)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/72.png?versionId=CAEQFRiBgICvypagxxciIDdhZWI1ODNjOGUzYTQ1YTFhOTNmMGQ3MzdlOGVlMTA2)

In [4]:
import torch
from torch import nn 
from torch.nn import functional as F

In [5]:
data=torch.ones(size=(10,3,229,229))
net = nn.Sequential(nn.Conv2d(3,6,3)
                    ,nn.ReLU(inplace=True)
                    ,nn.Conv2d(6,4,3)
                    ,nn.ReLU(inplace=True)
                    ,nn.MaxPool2d(2)
                    ,nn.Conv2d(4,16,5,stride=2,padding=1)
                    ,nn.ReLU(inplace=True)
                    ,nn.Conv2d(16,3,5,stride=3,padding=2)
                    ,nn.ReLU(inplace=True)
                    ,nn.MaxPool2d(2)
                   )
net(data).shape #Seq不是类，不需要实例化过程，可直接传入数据

torch.Size([10, 3, 9, 9])

二.注释掉线性层

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

data = torch.ones(size=(10, 3, 229, 229))

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        # block1
        self.conv1 = nn.Conv2d(3, 6, 3)  
        self.conv2 = nn.Conv2d(6, 4, 3)  
        self.pool1 = nn.MaxPool2d(2)             
        # block2
        self.conv3 = nn.Conv2d(4, 16, 5, stride=2, padding=1)  
        self.conv4 = nn.Conv2d(16, 3, 5, stride=3, padding=2)  
        self.pool2 = nn.MaxPool2d(2)                           
        
        # FC层
       # self.linear1 = nn.Linear(3*9*9, 256)
       #self.linear2 = nn.Linear(256, 256)
       #self.linear3 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool2(F.relu(self.conv4(x)))
       # x = x.view(-1, 3*9*9)  # 展平为 (batch_size, 192)
        
        #x = F.relu(self.linear1(F.dropout(x, p=0.5)))
        #x = F.relu(self.linear2(F.dropout(x, p=0.5)))
       # output = F.softmax(self.linear3(x), dim=1)
        
        return x

# 测试模型是否能正常运行
model = Model()
summary(model,input_size=(10,3,229,229))


Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 3, 9, 9]             --
├─Conv2d: 1-1                            [10, 6, 227, 227]         168
├─Conv2d: 1-2                            [10, 4, 225, 225]         220
├─MaxPool2d: 1-3                         [10, 4, 112, 112]         --
├─Conv2d: 1-4                            [10, 16, 55, 55]          1,616
├─Conv2d: 1-5                            [10, 3, 19, 19]           1,203
├─MaxPool2d: 1-6                         [10, 3, 9, 9]             --
Total params: 3,207
Trainable params: 3,207
Non-trainable params: 0
Total mult-adds (M): 251.17
Input size (MB): 6.29
Forward/backward pass size (MB): 44.89
Params size (MB): 0.01
Estimated Total Size (MB): 51.20